### Libraries

In [1]:
# data colelction and preprocessing
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import time 

# for data visualisation and statistical analysis
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set_style("white")
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from pylab import rcParams


%matplotlib inline

## Download  Url

In [23]:
def get_ads_urls():
    urls_list = []
    # define the basic url to crawl on
    basic_url = "https://autos.mercadolibre.com.ec/autos-camionetas/"
    supr_link = "_DisplayType_LF"
    pref_link = "_Desde_"
    con = [pref_link + str(sub) for sub in list(np.arange(1,501, 50))]
    con[0] = ""
    con = [ sub + supr_link for sub in con]
    
    for i in con:
        url = basic_url+str(i)
        r  = requests.get(url)
        data = r.text
        # transform it to bs object
        soup = BeautifulSoup(data, "lxml")
        # loop over page links
        for div in soup.findAll('div', {'class': 'ui-search-result__image'}):
            a = div.findAll('a')[0]
            urls_list.append(a.get('href'))
        time.sleep(5)
    df = pd.DataFrame(data={"url": urls_list})
    print(df.head())
    df.to_csv("./data/ads_urls_mercado.csv", sep=',',index=False)

In [24]:
get_ads_urls()

                                                 url
0  https://auto.mercadolibre.com.ec/MEC-426943093...
1  https://auto.mercadolibre.com.ec/MEC-427666865...
2  https://auto.mercadolibre.com.ec/MEC-426964946...
3  https://auto.mercadolibre.com.ec/MEC-427299436...
4  https://auto.mercadolibre.com.ec/MEC-427277173...


## Download Data de Url's

In [25]:
def Filter(string, substr): 
    return [str for str in string if
             any(sub in str for sub in substr)] 

def scrap_ad_data(ad_url):
    r = requests.get(ad_url)
    data = r.text
    soup = BeautifulSoup(data, "html.parser")
    target_component = soup.findAll(["li","span","div"],  {"class": ["specs-wrapper","price-tag-motors"]})
    #target_component_loc_text =soup.find("div", class_='vip-card').find("span").findAll(text=True)
    results=[]
    for i in target_component:
        results.append(''.join(i.findAll(text=True)).replace('\n',' '))
                         
    ult=[]
    for i in range(0,len(results)):
        ult.extend(results[i].split("  "))
    info= [i for i in ult if len(i)>0 ]    

    #info.extend(target_component_loc_text)
    info=Filter(info, ['Marca',"Modelo",'Kilómetros',"Año","U$S"]) 
    return info


In [26]:
def write_data_to_csv(data):
    with open("./data/output_Mercado.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerows(data)

In [27]:
def load_dataframe_info():
    urls_data = pd.read_csv("./data/ads_urls_mercado.csv")
    final_result = []
    for index, row in urls_data.iterrows():
        final_result.append(scrap_ad_data(row['url']))
    print('Scrapping data finished')
    #write_data_to_csv(final_result)
    return (final_result)

In [30]:
scrap=load_dataframe_info()
df = pd.DataFrame(scrap,columns=['carBrand','mileage',"carModel","year","price"])

display(df.head(10),df.shape)

,carBrand,mileage,carModel,year,price
0,U$S 13.000,None,None,None,None
1,Marca Chevrolet,Kilómetros 142000 km,Modelo Grand Vitara Sz 2.4,Año 2013,U$S 13.700
2,U$S 21.000,None,None,None,None
3,Marca Ford,Kilómetros 129600 km,Modelo Ecosport,Año 2008,U$S 12.500
4,Marca Kia,Kilómetros 112000 km,Modelo Rio r,Año 2014,U$S 12.000
5,U$S 6.500,None,None,None,None
6,Marca Kia,Kilómetros 56000 km,Modelo Sorento,Año 2005,U$S 15.950
7,Marca JAC,Kilómetros 45000 km,Modelo S3,Año 2019,U$S 16.850
8,U$S 12.900,None,None,None,None
9,U$S 16.500,None,None,None,None


(500, 5)

In [36]:
df.isnull().sum()

carBrand      0
mileage     325
carModel    325
year        325
price       325
dtype: int64

## Processing Data

In [ ]:
df = pd.read_csv("./data/output_Mercado.csv", sep=",", names=['carBrand','mileage',"carModel","year","price"], header=None,encoding="latin-1")
display(df.head(10),df.shape)

In [34]:
df_info=df.dropna(inplace=False).copy()
df_info["carBrand"]=df_info['carBrand'].map(lambda x: str(x)[6:])
df_info["carModel"]=df_info['carModel'].map(lambda x: str(x)[7:])
df_info['mileage']=df_info['mileage'].str.extract('(\d+)') 
df_info['year']=df_info['year'].str.extract('(\d+)') 
df_info['price']=df_info['price'].str.replace(".","").str.extract('(\d+)') 


df_info[["mileage","year","price"]]=df_info[["mileage","year","price"]].apply(pd.to_numeric)

display(df_info.head(3),df_info.shape)

,carBrand,mileage,carModel,year,price
1,Chevrolet,142000,Grand Vitara Sz 2.4,2013,13700
3,Ford,129600,Ecosport,2008,12500
4,Kia,112000,Rio r,2014,12000


(175, 5)

# Pruebas Unitarias 

In [ ]:
final_result=[]
for index, row in x.iterrows():
    final_result.append(scrap_ad_data(row['url']))
final_result

In [ ]:
x=scrap_ad_data('https://auto.mercadolibre.com.ec/MEC-427422595-ford-fiesta-power-1600-matriculado-2019-potente-veloz-2004-_JM#position=2&type=item&tracking_id=ad0c2b41-c8b5-484a-b682-99cb6fcb4c79')
x